In [8]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import np_utils

In [2]:
base = pd.read_csv('iris.csv')
base.head(2)

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa


In [3]:
previsores = base.iloc[:, 0:4].values
classe = base.iloc[:, 4].values

In [4]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [5]:
classe = labelencoder.fit_transform(classe)
classe[0:8]

array([0, 0, 0, 0, 0, 0, 0, 0])

In [6]:
classe_dummy = np_utils.to_categorical(classe)
# iris setosa     1 0 0
# iris virginica  0 1 0
# iris versicolor 0 0 1
classe_dummy[0:8]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [9]:
classificador = Sequential()

classificador.add(Dense(units = 8, activation = 'relu', 
                    kernel_initializer = 'normal', input_dim = 4))

classificador.add(Dropout(0.2))

classificador.add(Dense(units = 8, activation = 'relu', 
                    kernel_initializer = 'random_uniform'))

classificador.add(Dropout(0.3))

classificador.add(Dense(units = 3, activation = 'softmax'))

classificador.compile(optimizer = 'RMSprop', loss = 'categorical_crossentropy',
                      metrics = ['categorical_accuracy'])

classificador.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 40        
_________________________________________________________________
dropout (Dropout)            (None, 8)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 72        
_________________________________________________________________
dropout_1 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 27        
Total params: 139
Trainable params: 139
Non-trainable params: 0
_________________________________________________________________


In [10]:
classificador.fit(previsores, classe_dummy, batch_size = 10,
                  epochs = 1000)

Epoch 1/1000
15/15 [==============================] - 0s 929us/step - loss: 1.0991 - categorical_accuracy: 0.3333
Epoch 2/1000
15/15 [==============================] - 0s 904us/step - loss: 1.0924 - categorical_accuracy: 0.3267
Epoch 3/1000
15/15 [==============================] - 0s 1ms/step - loss: 1.0841 - categorical_accuracy: 0.3600
Epoch 4/1000
15/15 [==============================] - 0s 999us/step - loss: 1.0774 - categorical_accuracy: 0.3267
Epoch 5/1000
15/15 [==============================] - 0s 937us/step - loss: 1.0780 - categorical_accuracy: 0.3400
Epoch 6/1000
15/15 [==============================] - 0s 1ms/step - loss: 1.0657 - categorical_accuracy: 0.3733
Epoch 7/1000
15/15 [==============================] - 0s 900us/step - loss: 1.0529 - categorical_accuracy: 0.3533
Epoch 8/1000
15/15 [==============================] - 0s 933us/step - loss: 1.0507 - categorical_accuracy: 0.3533
Epoch 9/1000
15/15 [==============================] - 0s 933us/step - loss: 1.0519 - categor

In [11]:
classificador_json = classificador.to_json()
with open('classificador_iris.json', 'w') as json_file:
    json_file.write(classificador_json)

In [12]:
classificador.save_weights('classificador_iris.h5')

In [13]:
import numpy as np
import pandas as pd
from keras.models import model_from_json

In [14]:
arquivo = open('classificador_iris.json', 'r')
estrutura_rede = arquivo.read()
arquivo.close()

In [15]:
classificador = model_from_json(estrutura_rede)
classificador.load_weights('classificador_iris.h5')

In [16]:
novo = np.array([[4.9, 3, 1.4, 0.2]])
previsao = classificador.predict(novo)
previsao

array([[0.91335535, 0.08313846, 0.00350622]], dtype=float32)

In [17]:
previsao = (previsao > 0.5)
previsao

array([[ True, False, False]])

In [18]:
if previsao[0][0] == True and previsao[0][1] == False and previsao[0][2] == False:
    print('Iris setosa')
elif previsao[0][0] == False and previsao[0][1] == True and previsao[0][2] == False:
    print('Iris virginica')
elif previsao[0][0] == False and previsao[0][1] == False and previsao[0][2] == True:
    print('Iris versicolor')

Iris setosa
